# Computer Assignment 2: 2D Convolution and Image Noise Removal
## Alaqian Zafar - aaz7118

In [133]:
import numpy as np
import matplotlib.pyplot as plt

### **Problem 1** (Implementing 2D convolution in Python)
#### (a) Write 2D convolution function `conv2`, which implements "same"-padding convolution. For simplicity, you can assume the filter has an odd length in both vertical and horizontal directions. The input image should be grayscale. Assume pixel values outside the image are zero.

In [132]:
def conv2(image, filtr):
    result = np.empty([image.size]) # initialize result matrix
    pad_thick = int((len(filtr) - 1)/2) # padding thickness assuming square filter of odd legnth
    image = np.pad(image, pad_thick) # pad the image with zeros
    filtr = np.flip(filtr) # flip the filter
    for m in range(result.size[0]):
        for n in range(result.size[1]):
            result[m,n] = np.sum(np.multiply(image[m:m+pad_thick,n:n_pad_thick],filter))
    return result

#### (b) Write a function `plot filtering` that uses your conv2 function to filter a given input image with a given input filter and plots the following:
- The original input image and filtered image
- The log-magnitude spectrum of the original image, filter, and output image.

#### Use a grayscale colormap and a colorbar for each plot. For your filter-response, use an FFT size equal to the size of your image.

In [ ]:
def plot_image(image, title):
    plt.figure()
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.colorbar()

def plot_filtering(image, filtr):
    plot_image(